## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Alfred,ZA,-33.5906,26.8910,65.17,78,48,14.81,scattered clouds
1,1,Taguatinga,BR,-12.3857,-46.5711,91.90,18,0,6.06,clear sky
2,2,Puerto Ayora,EC,-0.7393,-90.3518,73.36,90,96,8.99,overcast clouds
3,3,Jamestown,US,42.0970,-79.2353,73.06,35,0,8.05,clear sky
4,4,Kovernino,RU,57.1253,43.8142,50.54,55,82,5.46,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Port Elizabeth,ZA,-33.9180,25.5701,75.09,75,0,4.61,clear sky
6,6,Namie,JP,37.4833,141.0000,77.72,93,100,5.41,overcast clouds
8,8,Kapaa,US,22.0752,-159.3190,77.32,83,40,12.66,scattered clouds
10,10,Takoradi,GH,4.8845,-1.7554,76.82,82,51,10.09,broken clouds
13,13,Warrington,GB,53.3925,-2.5802,76.87,49,0,10.36,clear sky


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 243 entries, 5 to 681
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              243 non-null    int64  
 1   City                 243 non-null    object 
 2   Country              243 non-null    object 
 3   Lat                  243 non-null    float64
 4   Lng                  243 non-null    float64
 5   Max Temp             243 non-null    float64
 6   Humidity             243 non-null    int64  
 7   Cloudiness           243 non-null    int64  
 8   Wind Speed           243 non-null    float64
 9   Current Description  243 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 20.9+ KB


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Port Elizabeth,ZA,75.09,clear sky,-33.9180,25.5701,
6,Namie,JP,77.72,overcast clouds,37.4833,141.0000,
8,Kapaa,US,77.32,scattered clouds,22.0752,-159.3190,
10,Takoradi,GH,76.82,broken clouds,4.8845,-1.7554,
13,Warrington,GB,76.87,clear sky,53.3925,-2.5802,
17,Nantucket,US,84.25,clear sky,41.2835,-70.0995,
20,Bengkulu,ID,75.20,overcast clouds,-3.8004,102.2655,
21,Atuona,PF,77.07,scattered clouds,-9.8000,-139.0333,
28,Kavieng,PG,81.68,overcast clouds,-2.5744,150.7967,
32,Ribeira Grande,PT,77.40,scattered clouds,38.5167,-28.7000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# Fill empty values with NaN value to make dropna work in next step.
hotel_df2=hotel_df.mask(hotel_df=="")
hotel_df2

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Port Elizabeth,ZA,75.09,clear sky,-33.9180,25.5701,39 On Nile Guest House
6,Namie,JP,77.72,overcast clouds,37.4833,141.0000,いこいの村なみえ
8,Kapaa,US,77.32,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
10,Takoradi,GH,76.82,broken clouds,4.8845,-1.7554,Raybow International Hotel
13,Warrington,GB,76.87,clear sky,53.3925,-2.5802,Villaggio Hotel
...,...,...,...,...,...,...,...
675,Aska,IN,80.47,broken clouds,19.6061,84.6725,Hotel Bhubaneswari
676,Tenenkou,ML,77.47,overcast clouds,14.4572,-4.9169,NaN
678,Lisakovsk,KZ,77.50,scattered clouds,52.5369,62.4936,Gostinitsa Lisakovsk
680,Kimbe,PG,75.25,overcast clouds,-5.5502,150.1429,Kimbe Bay Hotel


In [10]:
# 7. Drop the rows where there is no Hotel Name.

clean_hotel_df=hotel_df2.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Port Elizabeth,ZA,75.09,clear sky,-33.9180,25.5701,39 On Nile Guest House
6,Namie,JP,77.72,overcast clouds,37.4833,141.0000,いこいの村なみえ
8,Kapaa,US,77.32,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
10,Takoradi,GH,76.82,broken clouds,4.8845,-1.7554,Raybow International Hotel
13,Warrington,GB,76.87,clear sky,53.3925,-2.5802,Villaggio Hotel
...,...,...,...,...,...,...,...
669,Mazkeret Batya,IL,83.21,few clouds,31.8539,34.8386,Einav Holiday House בית הנופש עינב
675,Aska,IN,80.47,broken clouds,19.6061,84.6725,Hotel Bhubaneswari
678,Lisakovsk,KZ,77.50,scattered clouds,52.5369,62.4936,Gostinitsa Lisakovsk
680,Kimbe,PG,75.25,overcast clouds,-5.5502,150.1429,Kimbe Bay Hotel


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.


info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5, map_type='TERRAIN')
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))